In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from IPython.display import clear_output
import re

In [2]:
wd=webdriver.Chrome()
wd.get('https://www.hltv.org/stats/matches')

In [279]:

def next_page():
    element = wd.find_elements_by_class_name('pagination-next')
    element[0].click()
    
def if_next_page():
    element = wd.find_elements_by_class_name('pagination-next')
    next_page_exists=True
    if element[0].get_attribute('href') == 'https://www.hltv.org/stats/matches?offset=0':
        next_page_exists=False
    return next_page_exists
    
    
    return element[0].get_attribute('href')
    
def previous_page():
    element = wd.find_elements_by_class_name('pagination-prev')
    element[0].click()


def go_back_page():
    wd.back()

    
###dane_tabel    
def get_info_from_main_tabels():
    results_table = []
    tables = wd.find_elements_by_class_name('stats-table') 
    
    for table in tables:
        rows = table.find_elements_by_tag_name('tr')
        columns_names=get_columns_names(table)
    
    for row in rows:
        columns = row.find_elements_by_css_selector("td")
        temp = []
        for column in columns:
            temp.append(column.text)
        temp.append(get_link_summary(row))
        results_table.append(temp)
        df_out = pd.DataFrame(results_table)
        
        
    return df_out, columns_names



###nazwy kolumn
def get_columns_names(table):
    columns_names = []
    table_head = table.find_elements_by_tag_name('th')
    for head in table_head:
        columns_names.append(head.text)
    return columns_names
    
    
###pobierz link z glownej strony   
def get_link_summary(row):
    columns_link = row.find_elements_by_class_name('date-col')
    link=''
    for column in columns_link:
        if column.find_elements_by_css_selector('a'):
            temp2 = column.find_elements_by_css_selector('a')[0]
            link = temp2.get_attribute('href')
    return(link)
    
    
def in_database(df, last_element):
    m =  df.isin([last_element]).any()
    m = m.any()
    return m
 
    
## nowe dane do tabeli glownej

def get_new_data_to_main_table():
    df=pd.DataFrame()
    df_new, _ =  get_info_from_main_tabels()
    df = pd.concat([df, df_new])
    last_value=df.iloc[-1][5]

    while ~in_database(df_main_table, last_value):
        next_page()
        df_new, _ =  get_info_from_main_tabels()
        df = pd.concat([df, df_new])
        last_value = df.iloc[-1][5]
    df = process_main_table(df)
    df = df[~df['link'].isin(df_main_table['link'])]
    return df
    
    

    
    
# funkcje do czyszczenia tabeli glownej


#oczysczanie glownej tabeli
def process_main_table(df):
    df = df.dropna()
    columns_df_main = ['date', 'team1', 'team2', 'map', 'event', 'link']
    df.columns = columns_df_main
    df.reset_index(drop=True, inplace=True)
    df['score_team1'] = df.apply(lambda x: get_score_from_main_table(x['team1']), axis=1)
    df['team1'] = df.apply(lambda x: get_name_from_main_table(x['team1']), axis=1)
    df['score_team2'] = df.apply(lambda x: get_score_from_main_table(x['team2']), axis=1)
    df['team2'] = df.apply(lambda x: get_name_from_main_table(x['team2']), axis=1)
    df['date'] = pd.to_datetime(df['date'], format = '%d/%m/%y')
    return df

# wyczysc z nazwy wynik
def get_name_from_main_table(row_text):
    m=re.search('\((.*)\)' , row_text)
    if(m):
        row_text = row_text.replace(" "+m[0],'')
    return row_text    



# wez z nazwy wynik
def get_score_from_main_table(row_text):
    m=re.search('\((.*)\)' , row_text)
    if(m):
        score = m[1]
    else:
        score = np.nan
    return score

    
## FUNKCJE DO TABEL ZE SZCZEGOLAMI

def get_info_from_stats():
    list_info = {}
    list1 = wd.find_element_by_class_name('stats-match-maps') 
    list2 = list1.find_elements_by_class_name('stats-match-map')
    if list2:
        list_info['score_total'] = list2[0].text.split('\n')[0]
        list_info['match_type'] = list2[0].text.split('\n')[1]
        for idx, li in enumerate(list2):
            if(li.get_attribute("class") == 'col stats-match-map standard-box a-reset'):
                list_info['map_number'] = idx
    else:
        list_info['score_total'] = np.nan
        list_info['match_type'] = 'Best of 1'
        list_info['map_number'] = 1    
    return(list_info)

def get_match_info():
    dict_match_info={}
    match_info = wd.find_element_by_class_name('match-info-box-con')
    match_info = match_info.find_element_by_class_name('match-info-box')
    list_match_info = match_info.text.split("\n")
    dict_match_info['event'] = list_match_info[0]
    dict_match_info['date_time'] = list_match_info[1]
    dict_match_info['map_name'] = list_match_info[3]
    dict_match_info['score_team1'] = list_match_info[6]
    dict_match_info['score_team2'] = list_match_info[8]
    dict_match_info['team1'] = list_match_info[5]
    dict_match_info['team2'] = list_match_info[7]
    return(dict_match_info)

def get_match_info_details():
    dict_match_info_more = {}
    match_info = wd.find_element_by_class_name('match-info-box-con')
    match_info = match_info.find_elements_by_class_name('match-info-row')
    list_scores = match_info[0].find_elements_by_css_selector('span')
    dict_match_info_more['team1side'] = list_scores[2].get_attribute("class")
    dict_match_info_more['team1_score_1half']= list_scores[2].text
    dict_match_info_more['team2_score_1half']= list_scores[3].text
    dict_match_info_more['team1_score_2half']= list_scores[4].text
    dict_match_info_more['team2_score_2half']= list_scores[5].text
    dict_match_info_more['team1_rating'] = match_info[1].text.split("\n")[0].split(" ")[0]
    dict_match_info_more['team2_rating'] = match_info[1].text.split("\n")[0].split(" ")[2]
    dict_match_info_more['team1_first_kills'] = match_info[2].text.split("\n")[0].split(" ")[0]
    dict_match_info_more['team2_first_kills'] = match_info[2].text.split("\n")[0].split(" ")[2]
    dict_match_info_more['team1_clutches_won'] = match_info[3].text.split("\n")[0].split(" ")[0]
    dict_match_info_more['team2_clutches_won'] = match_info[3].text.split("\n")[0].split(" ")[2]
    return(dict_match_info_more)

def get_info_players():
    dict_players={}
    tables =wd.find_elements_by_class_name('stats-table') 
    for index_table,table in enumerate(tables):
        rows = table.find_elements_by_tag_name('tr')
        for index, row in enumerate(rows):
            temp = []
            columns = row.find_elements_by_css_selector("td")
            for column in columns:
                temp.append(column.text)
            if temp:
                dict_players.update(players_dict_create(temp, index, index_table+1))
    return (dict_players)

def players_dict_create(row, row_number, team_number):
    dict_to_return={}
    list_of_keys=['player_name','p_kills','p_asists','p_death','p_kast','p_ADR','p_rating']
    player_number = row_number
    dict_to_return[f'team{team_number}_{list_of_keys[0]}_{player_number}']=row[0]
    dict_to_return[f'team{team_number}_{list_of_keys[1]}_{player_number}']=row[1]
    dict_to_return[f'team{team_number}_{list_of_keys[2]}_{player_number}']=row[2]
    dict_to_return[f'team{team_number}_{list_of_keys[3]}_{player_number}']=row[3]
    dict_to_return[f'team{team_number}_{list_of_keys[4]}_{player_number}']=row[4]
    dict_to_return[f'team{team_number}_{list_of_keys[5]}_{player_number}']=row[6]
    dict_to_return[f'team{team_number}_{list_of_keys[6]}_{player_number}']=row[8]
    return dict_to_return
        






In [12]:
#tutaj rozwojewe funkcje

In [312]:
len(df_main_table.index)

65850

In [ ]:
df = pd.DataFrame()
number_of_rows = len(df_main_table.index)

for index, row in df_main_table.iterrows():
    dict_temp={}
    link = row['link']
    wd.get(link)
    dict_temp = get_info_from_stats()
    dict_temp.update(get_match_info())
    dict_temp.update(get_match_info_details())
    dict_temp.update(get_info_players())
    df = df.append(dict_temp,ignore_index=True)
    print(f'{index}/{number_of_rows}')
    clear_output(wait=True)
    
 
    

50/65850


In [308]:
df

,date_time,event,map_name,map_number,match_type,score_team1,score_team2,score_total,team1,team1_clutches_won,...,team2_p_rating_4,team2_p_rating_5,team2_player_name_1,team2_player_name_2,team2_player_name_3,team2_player_name_4,team2_player_name_5,team2_rating,team2_score_1half,team2_score_2half
0,2019-06-29 14:00,LOOT.BET Smack My Beach Cup,Map,1.0,Best of 1,12,16,NaN,DreamEaters,1,...,1.09,1.08,Farlig,roeJ,TOBIZ,HooXi,Remoy,1.23,7,9
1,2019-06-29 12:15,Good Game League 2019 Closed Qualifier,Map,2.0,Best of 3,7,16,0 - 2,BLUEJAYS,1,...,0.89,0.86,kalinka,redhide,def,shalfey,balblna,1.21,12,4
2,2019-06-29 12:15,Good Game League 2019 Closed Qualifier,Map,1.0,Best of 3,5,16,0 - 2,BLUEJAYS,0,...,1.19,1.00,kalinka,def,balblna,redhide,shalfey,1.35,11,5
3,2019-06-29 12:10,Good Game League 2019 Closed Qualifier,Map,2.0,Best of 3,13,16,0 - 2,OFFSET,0,...,0.81,0.67,NEEX,Patitek,Luz,EXUS,nawrot,1.13,7,9
4,2019-06-29 12:10,Good Game League 2019 Closed Qualifier,Map,1.0,Best of 3,19,22,0 - 2,OFFSET,1,...,0.99,0.83,Patitek,EXUS,NEEX,nawrot,Luz,1.09,10,5
5,2019-06-29 12:10,Good Game League 2019 Closed Qualifier,Map,2.0,Best of 3,16,12,2 - 0,Movistar Riders,1,...,0.91,0.70,kylar,hades,avis,MOLSI,h8me,0.98,8,4
6,2019-06-29 12:10,Good Game League 2019 Closed Qualifier,Map,1.0,Best of 3,16,7,2 - 0,Movistar Riders,1,...,0.86,0.71,h8me,MOLSI,avis,hades,kylar,0.91,4,3
7,2019-06-29 11:30,alpenScene Premiership Season 3 Finals,Map,2.0,Best of 3,16,19,1 - 2,WarKidZ,2,...,0.73,0.67,az3ry,ND,dannyK,rabtus,yuMaz,0.92,4,11
8,2019-06-29 11:30,alpenScene Premiership Season 3 Finals,Map,1.0,Best of 3,16,10,1 - 2,WarKidZ,3,...,0.78,0.77,rabtus,yuMaz,az3ry,dannyK,ND,1.01,6,4
9,2019-06-29 11:10,GG.BET Chicago Invitational - IEM Chicago 2019...,Map,2.0,Best of 3,16,13,2 - 0,Heroic,1,...,0.80,0.73,v1c7oR,bubble,blocker,poizon,SHiPZ,0.90,6,7


In [309]:
df = df.append(dict_temp, ignore_index=True)

In [303]:
df=pd.DataFrame()
df = df.append(dict_temp, ignore_index=True)

In [100]:
df_testowe = get_new_data_to_main_table()

C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

In [101]:
df_testowe

,date,team1,team2,map,event,link,score_team1,score_team2
0,2019-06-29,DreamEaters,Copenhagen Flames,Overpass,LOOT.BET Smack My Beach Cup,https://www.hltv.org/stats/matches/mapstatsid/...,12,16
1,2019-06-29,BLUEJAYS,Dracarys,Inferno,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,7,16
2,2019-06-29,BLUEJAYS,Dracarys,Mirage,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,5,16
3,2019-06-29,Izako Boars,OFFSET,Vertigo,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,13
4,2019-06-29,OFFSET,Izako Boars,Inferno,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,19,22
5,2019-06-29,Movistar Riders,piratesports,Inferno,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,12
6,2019-06-29,piratesports,Movistar Riders,Mirage,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,7,16
7,2019-06-29,plan-B,WarKidZ,Dust2,alpenScene Premiership Season 3 Finals,https://www.hltv.org/stats/matches/mapstatsid/...,19,16
8,2019-06-29,WarKidZ,plan-B,Nuke,alpenScene Premiership Season 3 Finals,https://www.hltv.org/stats/matches/mapstatsid/...,16,10
9,2019-06-29,Heroic,Windigo,Overpass,GG.BET Chicago Invitational - IEM Chicago 2019...,https://www.hltv.org/stats/matches/mapstatsid/...,16,13


In [70]:
df_main_table = pd.read_pickle("main_table.pkl")

In [44]:
df_main_table.sort_values('date', ascending=False)

,date,team1,team2,map,event,link,score_team1,score_team2
65732,2019-12-06,pro100,expert,Inferno,ESEA MDL Season 31 Europe,https://www.hltv.org/stats/matches/mapstatsid/...,16,1
65742,2019-12-06,TeamOne,ATK,Inferno,Americas Minor North America Closed Qualifier ...,https://www.hltv.org/stats/matches/mapstatsid/...,13,16
65749,2019-12-06,Movistar Riders,ALTERNATE aTTaX,Mirage,ESEA MDL Season 31 Europe,https://www.hltv.org/stats/matches/mapstatsid/...,16,9
65748,2019-12-06,Vitality,expert,Inferno,ESEA MDL Season 31 Europe,https://www.hltv.org/stats/matches/mapstatsid/...,16,5
65747,2019-12-06,AGO,x-kom,Train,ESEA MDL Season 31 Europe,https://www.hltv.org/stats/matches/mapstatsid/...,16,8
65746,2019-12-06,OpTic,Sprout,Mirage,Europe Minor Closed Qualifier - StarLadder Maj...,https://www.hltv.org/stats/matches/mapstatsid/...,10,16
65745,2019-12-06,Sprout,OpTic,Nuke,Europe Minor Closed Qualifier - StarLadder Maj...,https://www.hltv.org/stats/matches/mapstatsid/...,11,16
65744,2019-12-06,OpTic,Sprout,Dust2,Europe Minor Closed Qualifier - StarLadder Maj...,https://www.hltv.org/stats/matches/mapstatsid/...,3,16
65743,2019-12-06,ATK,TeamOne,Overpass,Americas Minor North America Closed Qualifier ...,https://www.hltv.org/stats/matches/mapstatsid/...,12,16
65741,2019-12-06,TeamOne,ATK,Train,Americas Minor North America Closed Qualifier ...,https://www.hltv.org/stats/matches/mapstatsid/...,16,6


In [187]:
df_main_table.iloc[-1]['link']

'https://www.hltv.org/stats/matches/mapstatsid/87919/movistar-riders-vs-g2'

In [66]:
df_main = pd.read_pickle('part1.pkl')

In [67]:
df = process_main_table(df_main)

C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

In [68]:
df.to_pickle('main_table.pkl')

9    2019-06-15
11   2019-06-15
4    2019-06-15
5    2019-06-15
6    2019-06-15
7    2019-06-15
8    2019-06-15
10   2019-06-15
12   2019-06-15
2    2019-06-15
13   2019-06-15
14   2019-06-15
20   2019-06-15
19   2019-06-15
18   2019-06-15
17   2019-06-15
3    2019-06-15
1    2019-06-15
15   2019-06-15
16   2019-06-15
1    2019-06-14
2    2019-06-14
3    2019-06-14
4    2019-06-14
5    2019-06-14
6    2019-06-14
49   2019-06-14
8    2019-06-14
9    2019-06-14
10   2019-06-14
        ...    
14   2012-10-09
25   2012-10-09
12   2012-10-09
11   2012-10-09
13   2012-10-09
26   2012-10-09
27   2012-10-08
29   2012-10-08
30   2012-10-08
31   2012-10-08
32   2012-10-08
33   2012-10-08
34   2012-10-08
35   2012-10-08
28   2012-10-08
43   2012-10-07
47   2012-10-07
46   2012-10-07
45   2012-10-07
44   2012-10-07
40   2012-10-07
42   2012-10-07
41   2012-10-07
39   2012-10-07
38   2012-10-07
36   2012-10-07
37   2012-10-07
48   2012-10-04
49   2012-10-04
50   2012-10-04
Name: 0, Length: 65850, 

In [199]:
wd=webdriver.Chrome()
wd.get('https://www.hltv.org/stats/matches')


In [ ]:





for i in range(ile):
    df, columns = get_info_from_tabels()
    next_page()
    df_all = pd.concat([df_all, df])
    print(f"{i} / {ile}")
    df_ile =pd.DataFrame([wd.find_element_by_class_name('pagination-data').text])
    df_ile.to_csv('ile_mamy.csv')
    clear_output(wait=True)

In [236]:
df=pd.DataFrame()

In [238]:
df_new, _ =  get_info_from_tabels()







next_page()
df = pd.concat([df, df_new])

In [85]:
df=pd.DataFrame()
df_new, _ =  get_info_from_main_tabels()
df = pd.concat([df, df_new])
last_value=df.iloc[-1][5]

while ~in_database(df_main_table, last_value):
    next_page()
    df_new, _ =  get_info_from_main_tabels()
    df = pd.concat([df, df_new])
    last_value = df.iloc[-1][5]

In [86]:
df = process_main_table(df)

C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

In [90]:
dana = df.iloc[-1]['link']

In [98]:
df[~df['link'].isin(df_main_table['link'])]

,date,team1,team2,map,event,link,score_team1,score_team2
0,2019-06-29,BLUEJAYS,Dracarys,Inferno,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,7,16
1,2019-06-29,BLUEJAYS,Dracarys,Mirage,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,5,16
2,2019-06-29,Izako Boars,OFFSET,Vertigo,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,13
3,2019-06-29,OFFSET,Izako Boars,Inferno,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,19,22
4,2019-06-29,Movistar Riders,piratesports,Inferno,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,12
5,2019-06-29,piratesports,Movistar Riders,Mirage,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,7,16
6,2019-06-29,plan-B,WarKidZ,Dust2,alpenScene Premiership Season 3 Finals,https://www.hltv.org/stats/matches/mapstatsid/...,19,16
7,2019-06-29,WarKidZ,plan-B,Nuke,alpenScene Premiership Season 3 Finals,https://www.hltv.org/stats/matches/mapstatsid/...,16,10
8,2019-06-29,Heroic,Windigo,Overpass,GG.BET Chicago Invitational - IEM Chicago 2019...,https://www.hltv.org/stats/matches/mapstatsid/...,16,13
9,2019-06-29,Heroic,Windigo,Mirage,GG.BET Chicago Invitational - IEM Chicago 2019...,https://www.hltv.org/stats/matches/mapstatsid/...,16,12


In [93]:
df_main_table[df_main_table.date >= '2019-06-15']

,date,team1,team2,map,event,link,score_team1,score_team2
65800,2019-06-15,Asterion,mCon,Mirage,ESL Dutch Championship Summer 2019 Finals,https://www.hltv.org/stats/matches/mapstatsid/...,17,19
65801,2019-06-15,mCon,Asterion,Dust2,ESL Dutch Championship Summer 2019 Finals,https://www.hltv.org/stats/matches/mapstatsid/...,16,8
65802,2019-06-15,CR4ZY,Ancient,Inferno,DreamHack Open Summer 2019,https://www.hltv.org/stats/matches/mapstatsid/...,12,16
65803,2019-06-15,Aetas x Strijders,Defusekids,Nuke,ESL Dutch Championship Summer 2019 Finals,https://www.hltv.org/stats/matches/mapstatsid/...,3,16
65804,2019-06-15,Defusekids,Aetas x Strijders,Dust2,ESL Dutch Championship Summer 2019 Finals,https://www.hltv.org/stats/matches/mapstatsid/...,16,6
65805,2019-06-15,Winstrike,AVANGAR,Vertigo,DreamHack Open Summer 2019,https://www.hltv.org/stats/matches/mapstatsid/...,14,16
65806,2019-06-15,OFFSET,GTZ-Bulls,Train,Master League Portugal Season III,https://www.hltv.org/stats/matches/mapstatsid/...,16,6
65807,2019-06-15,GTZ-Bulls,OFFSET,Inferno,Master League Portugal Season III,https://www.hltv.org/stats/matches/mapstatsid/...,5,16
65808,2019-06-15,5balls,Unique,Train,LOOT.BET Smack My Beach CIS Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,13,16
65809,2019-06-15,Unique,5balls,Dust2,LOOT.BET Smack My Beach CIS Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,4


In [12]:
element = df_main_table.iloc[-1]['link']

In [14]:
element

'https://www.hltv.org/stats/matches/mapstatsid/87919/movistar-riders-vs-g2'

In [92]:
df[df['date'] >= '2019-06-15']

,date,team1,team2,map,event,link,score_team1,score_team2
0,2019-06-29,BLUEJAYS,Dracarys,Inferno,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,7,16
1,2019-06-29,BLUEJAYS,Dracarys,Mirage,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,5,16
2,2019-06-29,Izako Boars,OFFSET,Vertigo,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,13
3,2019-06-29,OFFSET,Izako Boars,Inferno,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,19,22
4,2019-06-29,Movistar Riders,piratesports,Inferno,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16,12
5,2019-06-29,piratesports,Movistar Riders,Mirage,Good Game League 2019 Closed Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,7,16
6,2019-06-29,plan-B,WarKidZ,Dust2,alpenScene Premiership Season 3 Finals,https://www.hltv.org/stats/matches/mapstatsid/...,19,16
7,2019-06-29,WarKidZ,plan-B,Nuke,alpenScene Premiership Season 3 Finals,https://www.hltv.org/stats/matches/mapstatsid/...,16,10
8,2019-06-29,Heroic,Windigo,Overpass,GG.BET Chicago Invitational - IEM Chicago 2019...,https://www.hltv.org/stats/matches/mapstatsid/...,16,13
9,2019-06-29,Heroic,Windigo,Mirage,GG.BET Chicago Invitational - IEM Chicago 2019...,https://www.hltv.org/stats/matches/mapstatsid/...,16,12


In [20]:
df.iloc[-1]

date                                         2019-06-15 00:00:00
team1                                                  GTZ-Bulls
team2                                                     OFFSET
map                                                      Inferno
event                          Master League Portugal Season III
link           https://www.hltv.org/stats/matches/mapstatsid/...
score_team1                                                    5
score_team2                                                   16
Name: 549, dtype: object

In [180]:
df_new = df_main_table

In [181]:
df_new = process_main_table(df_new)

C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

In [183]:
df_new.to_pickle('main_table.pkl')

In [175]:
columns_df_main = ['date', 'team1', 'team2', 'map', 'event', 'link']
df_new.columns = columns_df_main

In [176]:
df_new.reset_index(drop=True, inplace=True)
df_new['score_team1'] = df_new.apply(lambda x: get_score_from_main_table(x['team1']), axis=1)
df_new['team1'] =df_new.apply(lambda x: get_name_from_main_table(x['team1']), axis=1)
df_new['score_team2'] = df_new.apply(lambda x: get_score_from_main_table(x['team2']), axis=1)
df_new['team2'] =df_new.apply(lambda x: get_name_from_main_table(x['team2']), axis=1)







C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [178]:
pd.to_datetime(df_new['date'])

0       2012-11-10
1       2012-11-10
2       2012-11-10
3       2012-10-10
4       2012-10-10
5       2012-10-10
6       2012-10-10
7       2012-10-10
8       2012-10-10
9       2012-09-10
10      2012-09-10
11      2012-09-10
12      2012-09-10
13      2012-09-10
14      2012-09-10
15      2012-09-10
16      2012-09-10
17      2012-09-10
18      2012-09-10
19      2012-09-10
20      2012-09-10
21      2012-09-10
22      2012-09-10
23      2012-09-10
24      2012-09-10
25      2012-09-10
26      2012-08-10
27      2012-08-10
28      2012-08-10
29      2012-08-10
           ...    
65820   2019-06-14
65821   2019-06-14
65822   2019-06-14
65823   2019-06-14
65824   2019-06-14
65825   2019-06-14
65826   2019-06-14
65827   2019-06-14
65828   2019-06-14
65829   2019-06-14
65830   2019-06-14
65831   2019-06-14
65832   2019-06-14
65833   2019-06-14
65834   2019-06-14
65835   2019-06-14
65836   2019-06-14
65837   2019-06-14
65838   2019-06-14
65839   2019-06-14
65840   2019-06-14
65841   2019

In [139]:
df_new.iloc[5706]['link']

'https://www.hltv.org/stats/matches/mapstatsid/19600/playing-ducks-vs-space-soldiers'

In [160]:
text = df_new.iloc[5706][1]

In [161]:
print(text)
m=re.search('\((.*)\)' , text)

Playing Ducks


In [162]:
if m:
    print("jest")

In [151]:
m[0]

TypeError: 'NoneType' object is not subscriptable

In [143]:
text.replace(" "+m[0],'')

TypeError: 'NoneType' object is not subscriptable

In [150]:
m[0]

TypeError: 'NoneType' object is not subscriptable

In [61]:
columns = ['date', 'team1', 'team2', 'map', 'event', 'link']

In [163]:
def get_name_from_main_table(row_text):
    m=re.search('\((.*)\)' , row_text)
    if(m):
        row_text = row_text.replace(" "+m[0],'')
    return row_text


In [167]:
def get_score_from_main_table(row_text):
    m=re.search('\((.*)\)' , row_text)
    if(m):
        score = m[1]
    else:
        score = np.nan
    return score

In [168]:
df_new['score_team1'] = df_new.apply(lambda x: get_score_from_main_table(x['team2']), axis=1)


C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [165]:
df_new['score_team1'] = df_new.apply()
df_new['team1'] =df_new.apply(lambda x: get_name_from_main_table(x['team1']), axis=1)

C:\Users\Marcin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [169]:
df_new

,date,team1,team2,map,event,link,score_team1
0,11/10/12,NiP,HastaLaVista (3),Train_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,3
1,11/10/12,NiP,HastaLaVista (8),Inferno_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,8
2,11/10/12,Red Dead,mousesports (16),Inferno_se,ESL Pro Series Germany Winter Season 2012,https://www.hltv.org/stats/matches/mapstatsid/...,16
3,10/10/12,mousesports,Tt Dragons (8),Mirage_ce,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,8
4,10/10/12,Tt Dragons,mousesports (16),Dust2_se,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16
5,10/10/12,mousesports,ALTERNATE aTTaX (7),Dust2_se,ESL Pro Series Germany Winter Season 2012,https://www.hltv.org/stats/matches/mapstatsid/...,7
6,10/10/12,mTw,gamed!de (14),Nuke_se,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,14
7,10/10/12,gamed!de,mTw (16),Mirage_ce,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,16
8,10/10/12,gamed!de,mTw (13),Dust2_se,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,13
9,9/10/12,FMESPORTS,aliquo (13),Mirage_ce,ESWC 2012 UK Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...,13


In [101]:
ile = 1000 

for i in range(ile):
    df, columns = get_info_from_tabels()
    previous_page()
    df_all = pd.concat([df_all, df])
    print(f"{i} / {ile}")
    df_ile =pd.DataFrame([wd.find_element_by_class_name('pagination-data').text])
    df_ile.to_csv('ile_mamy.csv')
    clear_output(wait=True)

    

999 / 1000


In [99]:
df_ile

,0
0,50227 - 50276 of 66127


In [73]:
df, columns = get_info_from_tabels()

In [77]:
df_all = pd.concat([df_all,df])

In [102]:
df_all

,0,1,2,3,4,5
0,,None,None,None,None,None
1,11/10/12,NiP (16),HastaLaVista (3),Train_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...
2,11/10/12,NiP (16),HastaLaVista (8),Inferno_se,ESWC 2012 Nordic Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...
3,11/10/12,Red Dead (2),mousesports (16),Inferno_se,ESL Pro Series Germany Winter Season 2012,https://www.hltv.org/stats/matches/mapstatsid/...
4,10/10/12,mousesports (16),Tt Dragons (8),Mirage_ce,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...
5,10/10/12,Tt Dragons (7),mousesports (16),Dust2_se,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...
6,10/10/12,mousesports (16),ALTERNATE aTTaX (7),Dust2_se,ESL Pro Series Germany Winter Season 2012,https://www.hltv.org/stats/matches/mapstatsid/...
7,10/10/12,mTw (16),gamed!de (14),Nuke_se,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...
8,10/10/12,gamed!de (7),mTw (16),Mirage_ce,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...
9,10/10/12,gamed!de (16),mTw (13),Dust2_se,ESWC 2012 German Qualifier,https://www.hltv.org/stats/matches/mapstatsid/...


In [103]:
df_all.to_pickle('part1.pkl')

In [55]:
last_element = df.iloc[2]
last_element = last_element[5]
last_element = ''

m =  df.isin([last_element]).any()


print(m.any())



True


In [222]:
results_table = []
results_table2 = []
table =wd.find_element_by_class_name('stats-table') 
rows = table.find_elements_by_tag_name('tr')

for row in rows:

    temp = []
    temp2 = []
    columns = row.find_elements_by_css_selector("td")
    
    
    for column in columns:
        temp.append(column.text)
    
    columns_link = row.find_elements_by_class_name('date-col')
    for column in columns_link:
        if column.find_elements_by_css_selector('a'):
            temp2 = column.find_elements_by_css_selector('a')[0]
            temp2 = temp2.get_attribute('href')
       
        
    results_table.append(temp)
    results_table2.append(temp2)

In [103]:
go_back_page()

In [4]:
df=pd.DataFrame(results_table)

In [25]:

tables = wd.find_elements_by_class_name('stats-table')
for table in tables:
    table_head = table.find_elements_by_tag_name('th')

In [26]:
for head in table_head:
    print(head.text)

Date
Team1
Team2
Map
Event


In [229]:
pd.to_datetime(df.iloc[2][0])

Timestamp('2019-06-18 00:00:00')

In [238]:
text = df.iloc[5][2]

In [239]:
import re

In [242]:
m=re.search('\((.*?)\)' , text)

In [244]:
m[0]

'(22)'

In [250]:
for i in range(2,15):
    text = df.iloc[i][2]
    m=re.search('\((.*?)\)' , text)
    print(m[0])
    print(text.replace(m[0],"")[:-1])

(9)
DETONA
(4)
DETONA
(6)
forZe
(22)
cats
(16)
Nemiga
(16)
Nemiga
(16)
DreamEaters
(12)
OFFSET
(17)
DreamEaters
(17)
G2
(17)
G2
(11)
NRG
(16)
fnatic


In [7]:
wd.get(results_table2[1])

In [218]:


for link in results_table2:
    wd.get(link)
    testowo()
    
    
# for link in lista_linkow:
#     wd.get(link)

0 - 2
Best of 3
col stats-match-map standard-box a-reset inactive
7 - 16
Vertigo
col stats-match-map standard-box a-reset
7 - 16
Inferno
col stats-match-map standard-box a-reset inactive

col stats-match-map
0 - 2
Best of 3
col stats-match-map standard-box a-reset inactive
4 - 16
Dust2
col stats-match-map standard-box a-reset inactive
9 - 16
Mirage
col stats-match-map standard-box a-reset

col stats-match-map
0 - 2
Best of 3
col stats-match-map standard-box a-reset inactive
4 - 16
Dust2
col stats-match-map standard-box a-reset
9 - 16
Mirage
col stats-match-map standard-box a-reset inactive

col stats-match-map
0 - 2
Best of 3
col stats-match-map standard-box a-reset inactive
20 - 22
Mirage
col stats-match-map standard-box a-reset inactive
6 - 16
Inferno
col stats-match-map standard-box a-reset

col stats-match-map
0 - 2
Best of 3
col stats-match-map standard-box a-reset inactive
20 - 22
Mirage
col stats-match-map standard-box a-reset
6 - 16
Inferno
col stats-match-map standard-box a-re

In [118]:
go_back_page()

In [21]:
next_page()

In [216]:
results_table2=results_table2[1:]

In [217]:
results_table2

['https://www.hltv.org/stats/matches/mapstatsid/88172/astralis-vs-heroic',
 'https://www.hltv.org/stats/matches/mapstatsid/88167/cloud9-vs-detona',
 'https://www.hltv.org/stats/matches/mapstatsid/88163/cloud9-vs-detona',
 'https://www.hltv.org/stats/matches/mapstatsid/88171/cats-vs-forze',
 'https://www.hltv.org/stats/matches/mapstatsid/88168/forze-vs-cats',
 'https://www.hltv.org/stats/matches/mapstatsid/88162/kova-vs-nemiga',
 'https://www.hltv.org/stats/matches/mapstatsid/88158/kova-vs-nemiga',
 'https://www.hltv.org/stats/matches/mapstatsid/88164/offset-vs-dreameaters',
 'https://www.hltv.org/stats/matches/mapstatsid/88161/dreameaters-vs-offset',
 'https://www.hltv.org/stats/matches/mapstatsid/88155/offset-vs-dreameaters',
 'https://www.hltv.org/stats/matches/mapstatsid/88165/nrg-vs-g2',
 'https://www.hltv.org/stats/matches/mapstatsid/88159/nrg-vs-g2',
 'https://www.hltv.org/stats/matches/mapstatsid/88154/g2-vs-nrg',
 'https://www.hltv.org/stats/matches/mapstatsid/88157/grayhound-v

In [131]:
wd.current_url

'https://www.hltv.org/stats/matches?offset=50'

In [276]:
def get_info_players()
    dict_players={}
    tables =wd.find_elements_by_class_name('stats-table') 
    for index_table,table in enumerate(tables):
        rows = table.find_elements_by_tag_name('tr')
        for index, row in enumerate(rows):
            temp = []
            columns = row.find_elements_by_css_selector("td")
            for column in columns:
                temp.append(column.text)
            if temp:
                dict_players.update(players_dict_create(temp, index, index_table+1))
    return (dict_players)

In [275]:
def players_dict_create(row, row_number, team_number):
    dict_to_return={}
    list_of_keys=['player_name','p_kills','p_asists','p_death','p_kast','p_ADR','p_rating']
    player_number = row_number
    dict_to_return[f'team{team_number}_{list_of_keys[0]}_{player_number}']=row[0]
    dict_to_return[f'team{team_number}_{list_of_keys[1]}_{player_number}']=row[1]
    dict_to_return[f'team{team_number}_{list_of_keys[2]}_{player_number}']=row[2]
    dict_to_return[f'team{team_number}_{list_of_keys[3]}_{player_number}']=row[3]
    dict_to_return[f'team{team_number}_{list_of_keys[4]}_{player_number}']=row[4]
    dict_to_return[f'team{team_number}_{list_of_keys[5]}_{player_number}']=row[6]
    dict_to_return[f'team{team_number}_{list_of_keys[6]}_{player_number}']=row[8]
    return dict_to_return
        

In [189]:
match = wd.find_element_by_class_name('stats-match-maps')
score = match.find_element_by_class_name('stats-match-map-result-score')
score_text = score.text
type_of_match =  match.find_element_by_class_name('dynamic-map-name-short')
type_of_match_text = type_of_match.text
print(score_text)
print(type_of_match_text)

2 - 1



In [277]:
dict_players

{'team1_player_name_1': 'TENZKI',
 'team1_p_kills_1': '21',
 'team1_p_asists_1': '6',
 'team1_p_death_1': '20',
 'team1_p_kast_1': '63.3%',
 'team1_p_ADR_1': '83.7',
 'team1_p_rating_1': '1.09',
 'team1_player_name_2': 'hallzerk',
 'team1_p_kills_2': '22',
 'team1_p_asists_2': '2',
 'team1_p_death_2': '18',
 'team1_p_kast_2': '60.0%',
 'team1_p_ADR_2': '71.4',
 'team1_p_rating_2': '1.05',
 'team1_player_name_3': 'cromen',
 'team1_p_kills_3': '18',
 'team1_p_asists_3': '6',
 'team1_p_death_3': '21',
 'team1_p_kast_3': '66.7%',
 'team1_p_ADR_3': '70.2',
 'team1_p_rating_3': '0.98',
 'team1_player_name_4': 'RUBINO',
 'team1_p_kills_4': '18',
 'team1_p_asists_4': '6',
 'team1_p_death_4': '21',
 'team1_p_kast_4': '70.0%',
 'team1_p_ADR_4': '60.2',
 'team1_p_rating_4': '0.89',
 'team1_player_name_5': 'H4RR3',
 'team1_p_kills_5': '16',
 'team1_p_asists_5': '5',
 'team1_p_death_5': '22',
 'team1_p_kast_5': '66.7%',
 'team1_p_ADR_5': '58.5',
 'team1_p_rating_5': '0.81',
 'team2_player_name_1': 

In [190]:
type_of_match =  match.find_element_by_class_name('stats-match-map-result-mapname')

In [191]:
type_of_match.text


''

In [192]:
print(score_text)
print(type_of_match_text)

2 - 1



In [193]:
scores = match.find_elements_by_class_name('stats-match-map-result-score')

In [194]:
for score in scores:
    print(score.text)

2 - 1
7 - 16
16 - 11
16 - 12


In [195]:
type_of_match = match.find_elements_by_class_name('dynamic-map-name-short')

In [196]:
for match in type_of_match:
    print(match.text)

In [197]:
list__ = match.find_elements_by_class_name('stats-match-map')
for l in list__:
    string___= str(l.get_attribute("class"))
    print(string___)
    if 'inactive' in string___ :
        print('nieaktywne')
    else:
        print('aktywne')



In [159]:
match_info = wd.find_element_by_class_name('match-info-box-con')

In [160]:
event_name = match_info.find_element_by_class_name('text-ellipsis')

In [161]:
event_name.text

'Good Game League 2019 Closed Qualifier'

In [162]:
event_name = match_info.find_element_by_class_name('match-info-box')

In [163]:
lista_bla_bla = event_name.text

In [164]:
lista_bla_bla.split("\n")

['Good Game League 2019 Closed Qualifier',
 '2019-06-29 12:15',
 '',
 'Map',
 'Inferno',
 'BLUEJAYS',
 '7',
 'Dracarys',
 '16']

In [171]:

dict_match_info={}
match_info = wd.find_element_by_class_name('match-info-box-con')
match_info = match_info.find_element_by_class_name('match-info-box')
list_match_info = match_info.text.split("\n")
dict_match_info['event'] = list_match_info[0]
dict_match_info['date_time'] = list_match_info[1]
dict_match_info['team1'] = list_match_info[6]
dict_match_info['team3'] = list_match_info[8]

In [170]:
list_match_info

['Good Game League 2019 Closed Qualifier',
 '2019-06-29 12:15',
 '',
 'Map',
 'Inferno',
 'BLUEJAYS',
 '7',
 'Dracarys',
 '16']

In [172]:
dict_match_info

{'event': 'Good Game League 2019 Closed Qualifier',
 'date_time': '2019-06-29 12:15',
 'team1': '7',
 'team3': '16'}

In [216]:
def get_match_info_details()
    dict_match_info_more = {}
    match_info = wd.find_element_by_class_name('match-info-box-con')
    match_info = match_info.find_elements_by_class_name('match-info-row')
    list_scores = match_info[0].find_elements_by_css_selector('span')
    dict_match_info_more['team1side'] = list_scores[2].get_attribute("class")
    dict_match_info_more['team1_score_1half']= list_scores[2].text
    dict_match_info_more['team2_score_1half']= list_scores[3].text
    dict_match_info_more['team1_score_2half']= list_scores[4].text
    dict_match_info_more['team2_score_2half']= list_scores[5].text
    dict_match_info_more['team1_rating'] = match_info[1].text.split("\n")[0].split(" ")[0]
    dict_match_info_more['team2_rating'] = match_info[1].text.split("\n")[0].split(" ")[2]
    dict_match_info_more['team1_first_kills'] = match_info[2].text.split("\n")[0].split(" ")[0]
    dict_match_info_more['team2_first_kills'] = match_info[2].text.split("\n")[0].split(" ")[2]
    dict_match_info_more['team1_clutches_won'] = match_info[3].text.split("\n")[0].split(" ")[0]
    dict_match_info_more['team2_clutches_won'] = match_info[3].text.split("\n")[0].split(" ")[2]
    return(dict_match_info_more)

In [209]:
list_scores = match_info[0].find_elements_by_css_selector('span')

In [223]:
dict_match_info_more['team1side'] = list_scores[2].get_attribute("class")
dict_match_info_more['team1_score_1half']= list_scores[2].text
dict_match_info_more['team2_score_1half']= list_scores[3].text
dict_match_info_more['team1_score_2half']= list_scores[4].text
dict_match_info_more['team2_score_2half']= list_scores[5].text

In [234]:
dict_match_info_more['team1_rating'] = match_info[1].text.split("\n")[0].split(" ")[0]
dict_match_info_more['team2_rating'] = match_info[1].text.split("\n")[0].split(" ")[2]

In [236]:
dict_match_info_more['team1_first_kills'] = match_info[2].text.split("\n")[0].split(" ")[0]
dict_match_info_more['team2_first_kills'] = match_info[2].text.split("\n")[0].split(" ")[2]

In [238]:
dict_match_info_more['team1_clutches_won'] = match_info[3].text.split("\n")[0].split(" ")[0]
dict_match_info_more['team2_clutches_won'] = match_info[3].text.split("\n")[0].split(" ")[2]

In [239]:
dict_match_info_more

{'team1side': 'ct-color',
 'team1_score_1half': '6',
 'team2_score_1half': '9',
 'team1_score_2half': '10',
 'team2_score_2half': '5',
 'team1_rating': '1.21',
 'team2_rating': '0.89',
 'team1_first_kills': '21',
 'team2_first_kills': '9',
 'team1_clutches_won': '2',
 'team2_clutches_won': '3'}

In [204]:
for score in match_info[0].find_elements_by_css_selector('span'):
    print(score.get_attribute("class"))
    print(score.text)

won
16
lost
14
ct-color
6
t-color
9
t-color
10
ct-color
5


In [198]:
for event in event_name:
    print(event.text)

16 : 14 ( 6 : 9 ) ( 10 : 5 )
Breakdown
1.21 : 0.89
Team rating
21 : 9
First kills
2 : 3
Clutches won


In [186]:
event_name[0].find_element_by_class_name('won').text

'16'

In [187]:
event_name[0].find_element_by_class_name('lost').text

'14'

In [188]:
for event in event_name[0].find_elements_by_css_selector('span'):
    print(event.get_attribute("class"))
    print(event.text)

won
16
lost
14
t-color
9
ct-color
6
ct-color
7
t-color
8


In [201]:
lista1 = wd.find_element_by_class_name('stats-match-maps') 

In [202]:
lista2 = lista1.find_elements_by_class_name('stats-match-map')

In [213]:
def testowo():
    lista1 = wd.find_element_by_class_name('stats-match-maps') 
    lista2 = lista1.find_elements_by_class_name('stats-match-map')
    for li in lista2:
        print(li.text)
        print(li.get_attribute("class"))

In [214]:
testowo()

1 - 3
Best of 5
col stats-match-map standard-box a-reset inactive
9 - 16
Overpass
col stats-match-map standard-box a-reset inactive
14 - 16
Nuke
col stats-match-map standard-box a-reset inactive
9 - 16
Inferno
col stats-match-map standard-box a-reset


In [58]:
from selenium import webdriver

PROXY = "86.111.144.194:3128" # IP:PORT or HOST:PORT

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

chrome = webdriver.Chrome(options=chrome_options)
chrome.get("http://whatismyipaddress.com")

In [89]:
wd.find_element_by_class_name('pagination-data').text

'60377 - 60426 of 66127'

In [107]:
list_of_links  = df_all[5]

In [111]:
list_of_links = list_of_links.dropna()

In [115]:
ile_linkow = len(list_of_links)
for counter,link in enumerate(list_of_links):
    wd.get(link)
    print(f"{counter} / {ile_linkow} ")

0 / 65850 
1 / 65850 
2 / 65850 
3 / 65850 
4 / 65850 
5 / 65850 
6 / 65850 
7 / 65850 
8 / 65850 
9 / 65850 
10 / 65850 
11 / 65850 
12 / 65850 
13 / 65850 
14 / 65850 
15 / 65850 
16 / 65850 
17 / 65850 
18 / 65850 
19 / 65850 
20 / 65850 
21 / 65850 
22 / 65850 
23 / 65850 
24 / 65850 
25 / 65850 
26 / 65850 
27 / 65850 
28 / 65850 
29 / 65850 
30 / 65850 
31 / 65850 
32 / 65850 
33 / 65850 
34 / 65850 
35 / 65850 
36 / 65850 
37 / 65850 
38 / 65850 
39 / 65850 
40 / 65850 
41 / 65850 
42 / 65850 
43 / 65850 
44 / 65850 
45 / 65850 
46 / 65850 
47 / 65850 
48 / 65850 
49 / 65850 
50 / 65850 
51 / 65850 
52 / 65850 
53 / 65850 
54 / 65850 
55 / 65850 
56 / 65850 
57 / 65850 
58 / 65850 
59 / 65850 
60 / 65850 
61 / 65850 
62 / 65850 
63 / 65850 
64 / 65850 
65 / 65850 
66 / 65850 
67 / 65850 
68 / 65850 
69 / 65850 
70 / 65850 
71 / 65850 
72 / 65850 
73 / 65850 
74 / 65850 
75 / 65850 
76 / 65850 
77 / 65850 
78 / 65850 
79 / 65850 
80 / 65850 
81 / 65850 
82 / 65850 
83 / 65850 
84

639 / 65850 
640 / 65850 
641 / 65850 
642 / 65850 
643 / 65850 
644 / 65850 
645 / 65850 
646 / 65850 
647 / 65850 
648 / 65850 
649 / 65850 
650 / 65850 
651 / 65850 
652 / 65850 
653 / 65850 
654 / 65850 
655 / 65850 
656 / 65850 
657 / 65850 
658 / 65850 
659 / 65850 
660 / 65850 
661 / 65850 
662 / 65850 
663 / 65850 
664 / 65850 
665 / 65850 
666 / 65850 
667 / 65850 
668 / 65850 
669 / 65850 
670 / 65850 
671 / 65850 
672 / 65850 
673 / 65850 
674 / 65850 
675 / 65850 
676 / 65850 
677 / 65850 
678 / 65850 
679 / 65850 
680 / 65850 
681 / 65850 
682 / 65850 
683 / 65850 
684 / 65850 
685 / 65850 
686 / 65850 
687 / 65850 
688 / 65850 
689 / 65850 
690 / 65850 
691 / 65850 
692 / 65850 
693 / 65850 
694 / 65850 
695 / 65850 
696 / 65850 
697 / 65850 
698 / 65850 
699 / 65850 
700 / 65850 
701 / 65850 
702 / 65850 
703 / 65850 
704 / 65850 
705 / 65850 
706 / 65850 
707 / 65850 
708 / 65850 
709 / 65850 
710 / 65850 
711 / 65850 
712 / 65850 
713 / 65850 
714 / 65850 
715 / 65850 

1250 / 65850 
1251 / 65850 
1252 / 65850 
1253 / 65850 
1254 / 65850 
1255 / 65850 
1256 / 65850 
1257 / 65850 
1258 / 65850 
1259 / 65850 
1260 / 65850 
1261 / 65850 
1262 / 65850 
1263 / 65850 
1264 / 65850 
1265 / 65850 
1266 / 65850 
1267 / 65850 
1268 / 65850 
1269 / 65850 
1270 / 65850 
1271 / 65850 
1272 / 65850 
1273 / 65850 
1274 / 65850 
1275 / 65850 
1276 / 65850 
1277 / 65850 
1278 / 65850 
1279 / 65850 
1280 / 65850 
1281 / 65850 
1282 / 65850 
1283 / 65850 
1284 / 65850 
1285 / 65850 
1286 / 65850 
1287 / 65850 
1288 / 65850 
1289 / 65850 
1290 / 65850 
1291 / 65850 
1292 / 65850 
1293 / 65850 
1294 / 65850 
1295 / 65850 
1296 / 65850 
1297 / 65850 
1298 / 65850 
1299 / 65850 
1300 / 65850 
1301 / 65850 
1302 / 65850 
1303 / 65850 
1304 / 65850 
1305 / 65850 
1306 / 65850 
1307 / 65850 
1308 / 65850 
1309 / 65850 
1310 / 65850 
1311 / 65850 
1312 / 65850 
1313 / 65850 
1314 / 65850 
1315 / 65850 
1316 / 65850 
1317 / 65850 
1318 / 65850 
1319 / 65850 
1320 / 65850 
1321 /

KeyboardInterrupt: 

In [114]:
len(list_of_links)

65850

In [5]:
df_all = pd.read_pickle("part1.pkl")

In [11]:
df_links = df_all[5].dropna()
for link in df_links:
    wd.get(link)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=75.0.3770.100)


In [106]:
link_testowy = df_testowe.iloc[1]['link']

In [107]:
wd.get(link_testowy)